This notebook demonstrates how to do co-refinement of several datasets with *refnx*

In [ ]:
%matplotlib inline
from __future__ import print_function, division

from refnx.dataset import ReflectDataset
from refnx.analysis import Transform, CurveFitter, Objective, GlobalObjective, Parameter
from refnx.reflect import SLD, ReflectModel

import corner
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# load the data
e361 = ReflectDataset('e361r.txt')
e365 = ReflectDataset('e365r.txt')
e366 = ReflectDataset('e366r.txt')

In [ ]:
si = SLD(2.07, 'Si')
sio2 = SLD(3.47, 'SiO2')
polymer = SLD(2.0, 'polymer')
d2o = SLD(6.36, 'D2O')
h2o = SLD(-0.56, 'H2O')
cm3 = SLD(3.5, 'cm3.5')

sio2_l = sio2(30, 3)
sio2_l.thick.setp(vary=True, bounds=(1, 50))

# Each contrast uses the same polymer SLD. We account for contrast change
# using the volume fraction of solvent.
polymer_l = polymer(250, 3)
polymer_l.thick.setp(vary=True, bounds=(200, 300))
polymer_l.sld.real.setp(vary=True, bounds=(0.1, 2))
polymer_l.vfsolv.setp(vary=True, bounds=(0, 1))

# we're going to share the water/polymer roughness across all 3 datasets
water_poly_rough = Parameter(3, 'water_poly_rough')
d2o_l = d2o(0, water_poly_rough)
h2o_l = h2o(0, water_poly_rough)
cm3_l = cm3(0, water_poly_rough)

In [ ]:
structure361 = si | sio2_l | polymer_l | d2o_l
structure365 = si | sio2_l | polymer_l | cm3_l
structure366 = si | sio2_l | polymer_l | h2o_l

In [ ]:
model361 = ReflectModel(structure361)
model365 = ReflectModel(structure365)
model366 = ReflectModel(structure366)

model361.scale.setp(vary=True, bounds=(0.9, 1.1))
model361.bkg.setp(vary=True, bounds=(0.9e-8, 3e-5))
model365.scale.setp(vary=True, bounds=(0.9, 1.1))
model365.bkg.setp(vary=True, bounds=(0.9e-8, 3e-5))
model366.bkg.setp(vary=True, bounds=(0.9e-8, 3e-5))

In [ ]:
objective361 = Objective(model361, e361, transform=Transform('logY'))
objective365 = Objective(model365, e365, transform=Transform('logY'))
objective366 = Objective(model366, e366, transform=Transform('logY'))

global_objective = GlobalObjective([objective361, objective365, objective366])

In [ ]:
# create the fit instance
fitter = CurveFitter(global_objective, threads=4, nwalkers=100)

res = fitter.fit('differential_evolution')

In [ ]:
plt.scatter(e361.x, e361.y, label='e361', color='r')
plt.scatter(e365.x, e365.y, label='e365', color='g')
plt.scatter(e366.x, e366.y, label='e366', color='b')

plt.plot(e361.x, model361(e361.x, x_err=e361.x_err), label='fit')
plt.plot(e365.x, model365(e365.x, x_err=e365.x_err), label='fit')
plt.plot(e366.x, model366(e366.x, x_err=e366.x_err), label='fit')

plt.ylim(np.min(e361.y), 0.1)
plt.xlim(0.005, np.max(e361.x))
plt.xlabel('Q')
plt.yscale('log')
plt.ylabel('logR')
plt.legend()

In [ ]:
print(global_objective)

In [ ]:
res = fitter.sample(1000, nburn=50, nthin=1)

In [ ]:
a = corner.corner(fitter.sampler.flatchain)

In [ ]:
print(global_objective)